In [1]:
import numpy as np
kre= np.zeros((8,8,4))

In [2]:
kre[0], kre

(array([[0., 0., 0., 0.],
        [0., 0., 0., 0.],
        [0., 0., 0., 0.],
        [0., 0., 0., 0.],
        [0., 0., 0., 0.],
        [0., 0., 0., 0.],
        [0., 0., 0., 0.],
        [0., 0., 0., 0.]]),
 array([[[0., 0., 0., 0.],
         [0., 0., 0., 0.],
         [0., 0., 0., 0.],
         [0., 0., 0., 0.],
         [0., 0., 0., 0.],
         [0., 0., 0., 0.],
         [0., 0., 0., 0.],
         [0., 0., 0., 0.]],
 
        [[0., 0., 0., 0.],
         [0., 0., 0., 0.],
         [0., 0., 0., 0.],
         [0., 0., 0., 0.],
         [0., 0., 0., 0.],
         [0., 0., 0., 0.],
         [0., 0., 0., 0.],
         [0., 0., 0., 0.]],
 
        [[0., 0., 0., 0.],
         [0., 0., 0., 0.],
         [0., 0., 0., 0.],
         [0., 0., 0., 0.],
         [0., 0., 0., 0.],
         [0., 0., 0., 0.],
         [0., 0., 0., 0.],
         [0., 0., 0., 0.]],
 
        [[0., 0., 0., 0.],
         [0., 0., 0., 0.],
         [0., 0., 0., 0.],
         [0., 0., 0., 0.],
         [0., 0., 0., 0.]

In [3]:
import random
random.choice(range(5))

2

In [4]:
for i in range(0,5, 2):
    print(i)

0
2
4


In [12]:
import pygame
import numpy as np
import random

# Constants
WIDTH, HEIGHT = 8, 8
CELL_SIZE = 100
WIN_WIDTH, WIN_HEIGHT = WIDTH * CELL_SIZE, HEIGHT * CELL_SIZE
FPS = 30
OBSTACLES = []  # Obstacle coordinates
for i in range(5):
    OBSTACLES.append((random.randint(0,7), random.randint(0,7)))

START_POS = (0, 0)  # Start position
END_POS = (WIDTH - 1, HEIGHT - 1)  # End position
ACTIONS = ["UP", "DOWN", "LEFT", "RIGHT"]
EPSILON = 0.3  # Epsilon for exploration
LEARNING_RATE = 0.1
DISCOUNT_FACTOR = 0.9
EPISODES = 1000

# Initialize Pygame
pygame.init()
window = pygame.display.set_mode((WIN_WIDTH, WIN_HEIGHT))
clock = pygame.time.Clock()

# Q-Table
q_table = np.zeros((WIDTH, HEIGHT, len(ACTIONS)))

# Helper function to draw the grid and obstacles
def draw_grid():
    for x in range(0, WIN_WIDTH, CELL_SIZE):
        pygame.draw.line(window, (255, 255, 255), (x, 0), (x, WIN_HEIGHT))
    for y in range(0, WIN_HEIGHT, CELL_SIZE):
        pygame.draw.line(window, (255, 255, 255), (0, y), (WIN_WIDTH, y))
    for obstacle in OBSTACLES:
        pygame.draw.rect(window, (255, 0, 0), (obstacle[0] * CELL_SIZE, obstacle[1] * CELL_SIZE, CELL_SIZE, CELL_SIZE))

# Function to choose an action based on the Q-values with epsilon-greedy policy
def choose_action(state):
    if np.random.rand() < EPSILON:
        return random.choice(range(len(ACTIONS)))  # Explore
    else:
        return np.argmax(q_table[state])  # Exploit

# Main Q-learning algorithm
def q_learning():
    global EPSILON  # Declare EPSILON as global to modify its value
    for episode in range(EPISODES):
        print(episode)
        state = START_POS
        done = False

        while not done:
            action = choose_action(state)
            if action == 0:  # UP
                next_state = (state[0], max(0, state[1] - 1))
            elif action == 1:  # DOWN
                next_state = (state[0], min(HEIGHT - 1, state[1] + 1))
            elif action == 2:  # LEFT
                next_state = (max(0, state[0] - 1), state[1])
            else:  # RIGHT
                next_state = (min(WIDTH - 1, state[0] + 1), state[1])

            # Check for obstacles
            if next_state not in OBSTACLES:
                reward = -2  # Reward for moving to a valid state
            else:
                reward = -10  # Reward for moving to an invalid state

            # Update Q-value using the Bellman equation
            q_table[state][action] = (1 - LEARNING_RATE) * q_table[state][action] + \
                                     LEARNING_RATE * (reward + DISCOUNT_FACTOR * np.max(q_table[next_state]))

            state = next_state

            # Check if the agent reached the goal
            if state == END_POS:
                done = True

        # Reduce epsilon over time
        EPSILON = max(0.1, EPSILON * 0.99)

# Function to draw the agent on the grid
def draw_agent(pos):
    pygame.draw.rect(window, (0, 0, 255), (pos[0] * CELL_SIZE, pos[1] * CELL_SIZE, CELL_SIZE, CELL_SIZE))

# Game loop
def main():
    # here the main purpose of the q_learning() is to update the q_table
    # the q_table is used below.
    # the q_table with all zero values will be updated and used in the below code after q_learning() function
    q_learning()
    
    current_pos = START_POS

    running = True
    while running:
        window.fill((0, 0, 0))

        for event in pygame.event.get():
            if event.type == pygame.QUIT:
                running = False

        draw_grid()
        draw_agent(current_pos)

        # Move the agent based on the learned Q-values
        action = np.argmax(q_table[current_pos])
        print(q_table[current_pos], action)
        if action == 0:  # UP
            next_pos = (current_pos[0], max(0, current_pos[1] - 1))
        elif action == 1:  # DOWN
            next_pos = (current_pos[0], min(HEIGHT - 1, current_pos[1] + 1))
        elif action == 2:  # LEFT
            next_pos = (max(0, current_pos[0] - 1), current_pos[1])
        else:  # RIGHT
            next_pos = (min(WIDTH - 1, current_pos[0] + 1), current_pos[1])

        # Check for obstacles
        if next_pos not in OBSTACLES:
            current_pos = next_pos

        pygame.display.update()
        clock.tick(FPS)

if __name__ == "__main__":
    main()
    pygame.quit()


0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
27

In [6]:
q_table

array([[[-15.39568245, -15.28194022, -15.42826563, -15.28071024],
        [-14.98382711, -14.7753666 , -14.91416902, -14.77529347],
        [-14.42672051, -14.21973449, -14.26611779, -14.21772055],
        [-13.82963264, -13.60739333, -13.66455771, -13.60749068],
        [-12.94842586, -12.93305633, -12.97980109, -12.94036324],
        [-12.24139875, -12.19138043, -12.2156543 , -12.19413948],
        [-11.68604082, -11.4071989 , -11.51578937, -11.37343244],
        [-11.08923615, -11.02420655, -10.99835382, -11.00386688]],

       [[-14.79858913, -14.77302996, -15.07973808, -14.77211809],
        [-14.27575688, -14.2489571 , -14.33269923, -14.24660535],
        [-13.78567763, -13.64946069, -13.92557816, -13.64782471],
        [-13.0282197 , -12.98100293, -13.09946025, -12.98248797],
        [-12.3051229 , -12.23087472, -12.650673  , -12.2293057 ],
        [-11.96708113, -11.38457874, -11.54769935, -11.38425682],
        [-10.65782394, -10.63182188, -10.58582697, -10.4340315 ],
        